<a href="https://colab.research.google.com/github/LuisjAlarcon/Data-Driven-in-Aerospace-Eng.-Class/blob/main/Alarcon_Final_Project_MAE5020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir set_runway

In [2]:
!mkdir set_sky

In [6]:
#I create a directory and unzip my dataset
%cd /content/set_runway

!unzip dataset2.zip
%cd ..

/content/set_runway
Archive:  dataset2.zip
  inflating: ZUXJ02_1_8LDImage1.png  
  inflating: ZUXJ02_1_8LDImage3.png  
  inflating: ZUXJ02_1_9FNLImage2.png  
  inflating: ZUXJ02_1_9FNLImage4.png  
  inflating: ZUXJ02_1_9FNLImage5.png  
  inflating: ZUXJ02_1_9FNLImage6.png  
  inflating: ZUXJ02_1_9LDImage1.png  
  inflating: ZUXJ02_1_9LDImage3.png  
  inflating: 4AK606_1_1FNLImage1.png  
  inflating: 4AK606_1_1LDImage1.png  
  inflating: 4AK606_1_1LDImage2.png  
  inflating: 4AK606_1_1LDImage3.png  
  inflating: 4AK606_1_2FNLImage1.png  
  inflating: 4AK606_1_2LDImage1.png  
  inflating: 4AK606_1_2LDImage2.png  
  inflating: 4AK606_1_2LDImage3.png  
  inflating: 4AK606_1_3FNLImage1.png  
  inflating: 4AK606_1_3FNLImage2.png  
  inflating: 4AK606_1_3LDImage1.png  
  inflating: 4AK606_1_3LDImage2.png  
  inflating: 4AK606_1_4FNLImage1.png  
  inflating: 4AK606_1_4LDImage1.png  
  inflating: 4AK606_1_4LDImage2.png  
  inflating: 4AK606_1_5FNLImage1.png  
  inflating: 4AK606_1_5FNLImage2.pn

In [31]:
%cd /content/set_sky

!unzip sky3.zip
%cd ..

/content/set_sky
Archive:  sky3.zip
  inflating: sun-and-sky-above-clouds-view-from-plane-high-altitude-S3PMPT.jpg  
  inflating: tokyo3.jpg              
  inflating: tokyo-city-skyline-aerial-view-japan-laurent-davoust.jpg  
  inflating: uds-above-the-blue-sky-with-the-sun-shining-sky-nature-background-free-video.jpg  
  inflating: uds-above-the-blue-sky-with-the-sun-shining-sky-nature-background-free-video2.jpg  
  inflating: uds-above-the-blue-sky-with-the-sun-shining-sky-nature-background-free-video3.jpg  
  inflating: uds-sun-summer-day-time-lapse-above-blue-rays-shining-white-nature-259981980.jpg  
  inflating: uds-sun-summer-day-time-lapse-above-blue-shining-nature-background-263331258.jpg  
  inflating: unusual-aerial-view-of-sky-and-clouds-over-the-earth-suitable-as-background-DJ3C0H.jpg  
  inflating: y7dyaua86hv51.jpg       
  inflating: aec-aerial-intelligence-property-data.png  
  inflating: aerial_view.png         
  inflating: 1AijU0RBcrMwmJVLWNnuTlg.jpeg  
  inflating:

In [33]:
#I will erase the zip file
!rm -rf /content/set_runway/dataset2.zip
!rm -rf /content/set_sky/sky3.zip.zip

In [34]:
!mkdir dataset
!mkdir dataset/runway
!mkdir dataset/aerialview

mkdir: cannot create directory ‘dataset’: File exists


In [36]:
!ls /content/set_runway | wc -l
!ls /content/set_sky | wc -l

3987
1192


In [37]:
import shutil
carpeta_fuente = '/content/set_runway'
carpeta_destino = '/content/dataset/runway'

imagenes = os.listdir(carpeta_fuente)

for i, nombreimg in enumerate(imagenes):
  if i < 1190:
    shutil.copy(carpeta_fuente + '/' + nombreimg, carpeta_destino + '/' + nombreimg)

carpeta_fuente = '/content/set_sky'
carpeta_destino = '/content/dataset/aerialview'

imagenes = os.listdir(carpeta_fuente)

for i, nombreimg in enumerate(imagenes):
  if i < 1190:
    #Copia de la carpeta fuente a la destino
    shutil.copy(carpeta_fuente + '/' + nombreimg, carpeta_destino + '/' + nombreimg)



In [39]:
!ls /content/dataset/runway | wc -l
!ls /content/dataset/aerialview | wc -l

1190
1190


In [40]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

#I use the image generator, only to split my dataset, in other models could be used to improve the dataset
datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 30,
    validation_split=0.2 #20% para pruebas
)

#I generate the set of training and test
datagen_training = datagen.flow_from_directory('/content/dataset', target_size=(240,240),
                                                     batch_size=32, shuffle=True, subset='training')
datagen_test = datagen.flow_from_directory('/content/dataset', target_size=(240,240),
                                                     batch_size=32, shuffle=True, subset='validation')

Found 1904 images belonging to 3 classes.
Found 475 images belonging to 3 classes.


In [27]:
import tensorflow as tf
import tensorflow_hub as hub

url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
mobilenetv2 = hub.KerasLayer(url, input_shape=(240,240,1))

In [28]:
#I freeze the model that I downloaded
mobilenetv2.trainable = False

In [30]:
modelo = tf.keras.Sequential([
    mobilenetv2,
    tf.keras.layers.Dense(1, activation='softmax')
])

ValueError: Exception encountered when calling layer "keras_layer_3" (type KerasLayer).

in user code:

    File "/usr/local/lib/python3.10/dist-packages/tensorflow_hub/keras_layer.py", line 250, in call  *
        result = smart_cond.smart_cond(training,

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (4 total):
        * <tf.Tensor 'inputs:0' shape=(None, 240, 240, 1) dtype=float32>
        * False
        * False
        * 0.99
      Keyword arguments: {}
    
     Expected these arguments to match one of the following 4 option(s):
    
    Option 1:
      Positional arguments (4 total):
        * TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='inputs')
        * True
        * False
        * TensorSpec(shape=(), dtype=tf.float32, name='batch_norm_momentum')
      Keyword arguments: {}
    
    Option 2:
      Positional arguments (4 total):
        * TensorSpec(shape=(None, 224, 224, 3), d

In [ ]:
modelo.summary()